In [ ]:
# !pipenv install chromadb && pip install -q chromadb

In [ ]:
# !pipenv install langchain-chroma && pip install -q langchain-chroma

In [4]:
%run 1.Ollama_Models.ipynb

## Load Navigation Intents

In [30]:
from json import load

In [31]:
with open("./data/navigation/navigation_intents.json") as f:
    intents = load(f)

In [32]:
intents[0]

{'intent': 'navigate_to_login_page',
 'description': 'Directs the user to the login page to authenticate and access the application.',
 'parameters': {'app_id': 'integer'},
 'required': ['app_id'],
 'responses': {'web': 'https://qa.sim.guardhat.com/login_page',
  'android': 'ActivityLogin',
  'ios': 'ActivityLogin'}}

In [33]:
intents[-1]

{'intent': 'navigate_to_form_listing',
 'description': 'Navigates to the form list screen to view available forms for execution.',
 'parameters': {'app_id': 'integer'},
 'required': ['app_id'],
 'responses': {'web': 'https://qa.sim.guardhat.com/form_listing_view',
  'android': 'ActivityFormListing',
  'ios': 'ActivityFormListing'}}

## Create Chroma Index

In [34]:
from langchain_chroma import Chroma

In [35]:
from langchain_core.documents import Document

In [36]:
PERSIST_DIRECTORY = "./data/vector_db/chroma/"

In [37]:
documents = []

for index, item in enumerate(intents, start=1):
    doc = Document(page_content=item["description"], metadata={"id": index})
    documents.append(doc)

In [38]:
len(intents) == len(documents)

True

## Load / Create Vector DB

In [39]:
try:
    vectordb = Chroma(persist_directory=PERSIST_DIRECTORY, embedding_function=embeddings)
except:
    vectordb = Chroma.from_documents(documents=documents, embedding=embeddings, persist_directory=PERSIST_DIRECTORY)

### Test Retrieval

In [40]:
retriever = vectordb.as_retriever()

In [41]:
retriever.invoke("how do I access search worker?")

[Document(id='55a7fa1f-5523-44a7-b5b4-4cf306e4870a', metadata={'id': 38}, page_content='Navigates to the search worker screen for assigning workers to areas or products.'),
 Document(id='ef52f546-d968-40b3-b8c2-42258a91037a', metadata={'id': 5}, page_content='Navigates to a detailed view of a specific worker or event, including relevant metadata.'),
 Document(id='701520de-23ba-47b4-8149-96fbb11a81e7', metadata={'id': 15}, page_content='Takes the user to the user or product search screen to find specific entities.'),
 Document(id='6f6b8290-1de3-49e6-85e7-9117b2db5ca8', metadata={'id': 22}, page_content='Directs to the user or product filter screen to refine search results.')]